In [317]:

import os
import re
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
from dotenv import load_dotenv
load_dotenv()

True

In [318]:
# file path source & target
source_directory_path = "C:\Repos\Agentic_AI\QM_Practice\Langchain\Source\manage_hr_data_cuurent.sql"
target_directory_path = "C:\Repos\Agentic_AI\QM_Practice\Langchain\Target\manage_hr_data_baseline.sql"

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:3: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:3: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Quadrant\AppData\Local\Temp\ipykernel_5940\3476313463.py:2: SyntaxWarning: invalid escape sequence '\R'
  source_directory_path = "C:\Repos\Agentic_AI\QM_Practice\Langchain\Source\manage_hr_data_cuurent.sql"
C:\Users\Quadrant\AppData\Local\Temp\ipykernel_5940\3476313463.py:3: SyntaxWarning: invalid escape sequence '\R'
  target_directory_path = "C:\Repos\Agentic_AI\QM_Practice\Langchain\Target\manage_hr_data_baseline.sql"


In [319]:
def read_and_split_sql(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    pattern = r'(?<=;)\s*\n'  # Match semicolon followed by optional whitespace and newline
    sql_statements = re.split(pattern, content.strip())
    sql_statements = [chunk.strip() for chunk in sql_statements if chunk.strip()]
    return sql_statements

In [320]:
source_statements = read_and_split_sql(source_directory_path)
target_statements = read_and_split_sql(target_directory_path)


In [321]:
source_docs = [
    Document(
        page_content=stmt,
        metadata={"file_name": source_directory_path, "line_number": i + 1}
    )
    for i, stmt in enumerate(source_statements)
]

target_docs = [
    Document(
        page_content=stmt,
        metadata={"file_name": target_directory_path, "line_number": i + 1}
    )
    for i, stmt in enumerate(target_statements)
]
embeddings = OpenAIEmbeddings()
target_vectorstore = FAISS.from_documents(target_docs, embeddings)


In [322]:
# Track matched indices
matched_source_indices = set()
matched_target_indices = set()
mappings = []

# Match source to target
for source_idx, source_doc in enumerate(source_docs):
    if source_idx in matched_source_indices:
        continue
    # Retrieve the most similar target
    docs_with_scores = target_vectorstore.similarity_search_with_score(source_doc.page_content, k=1)
    if docs_with_scores:
        print(source_doc, '=========')
        target_doc, score = docs_with_scores[0]
        print(target_doc, score)
        cosine_similarity = 1 - (score ** 2) / 2
        print(cosine_similarity)
        if cosine_similarity >= 0.8:
            target_idx = target_doc.metadata['line_number'] - 1  # Adjust for 0-based index
            if target_idx not in matched_target_indices:
                # Check reverse match (target to source)
                source_for_target = FAISS.from_documents([source_doc], embeddings)
                reverse_docs = source_for_target.similarity_search_with_score(target_doc.page_content, k=1)
                if reverse_docs:
                    reverse_source_doc, reverse_score = reverse_docs[0]
                    rev_cosine_similarity = 1 - (reverse_score ** 2) / 2
                    if rev_cosine_similarity >= 0.8 and reverse_source_doc.metadata['line_number'] == source_doc.metadata['line_number']:
                        mappings.append((source_doc, target_doc))
                        matched_source_indices.add(source_idx)
                        matched_target_indices.add(target_idx)


page_content='CREATE OR REPLACE PROCEDURE HR.manage_hr_data AS 
BEGIN
    INSERT INTO employees VALUES (300, 'John', 'Doe', 'jdoe@example.com', '515-555-1234', SYSDATE, 'IT_PROG', 6000, 103, 60);' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 1} =========
page_content='CREATE OR REPLACE PROCEDURE HR.manage_hr_data AS 
BEGIN
    INSERT INTO employees VALUES (300, 'John', 'Doe', 'jdoe@example.com', '515-555-1234', SYSDATE, 'IT_PROG', 6000, 103, 60);' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Target\\manage_hr_data_baseline.sql', 'line_number': 1} 0.0
1.0
page_content='INSERT INTO employees VALUES (301, 'Jane', 'Smith', 'jsmith@example.com', '515-555-5678', SYSDATE, 'ST_CLERK', 4000, 120, 50);' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 2} =========
page_content='INSERT INTO employees (employee_id, first_n

In [323]:
# Match remaining targets to sources (bidirectional check)
for target_idx, target_doc in enumerate(target_docs):
    if target_idx in matched_target_indices:
        continue
    # Retrieve the most similar source
    source_vectorstore = FAISS.from_documents(source_docs, embeddings)
    docs_with_scores = source_vectorstore.similarity_search_with_score(target_doc.page_content, k=1)
    if docs_with_scores:
        
        source_doc, score = docs_with_scores[0]
        print(source_doc, score)
        cosine_similarity = 1 - (score ** 2) / 2
        print(cosine_similarity)
        if score >= 0.8:
            source_idx = source_doc.metadata['line_number'] - 1
            if source_idx not in matched_source_indices:
                # Check reverse match (source to target)
                target_vectorstore_temp = FAISS.from_documents([target_doc], embeddings)
                reverse_docs = target_vectorstore_temp.similarity_search_with_score(source_doc.page_content, k=1)
                if reverse_docs:
                    reverse_target_doc, reverse_score = reverse_docs[0]
                    rev_cosine_similarity = 1 - (reverse_score ** 2) / 2
                    if rev_cosine_similarity >= 0.8 and reverse_target_doc.metadata['line_number'] == target_doc.metadata['line_number']:
                        mappings.append((source_doc, target_doc))
                        matched_source_indices.add(source_idx)
                        matched_target_indices.add(target_idx)

page_content='INSERT INTO employees VALUES (303, 'Mike', 'Johnson', 'mikej@example.com', '515-555-2222', SYSDATE, 'IT_PROG', 7500, 103, 60);' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 4} 0.14630084
0.98929805
page_content='DELETE FROM employees WHERE department_id = 30;' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 25} 0.11725505
0.9931256
page_content='UPDATE employees SET salary = salary * 1.02 WHERE job_id = 'IT_PROG';' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 14} 0.22248244
0.9752508
page_content='INSERT INTO jobs VALUES ('SEC_ANA', 'Security Analyst', 5000, 12000);' metadata={'file_name': 'C:\\Repos\\Agentic_AI\\QM_Practice\\Langchain\\Source\\manage_hr_data_cuurent.sql', 'line_number': 51} 0.22652668
0.9743428
page_content='INSERT INTO employe

In [324]:
print(matched_source_indices)
print(matched_target_indices)

{0, 1, 10, 11, 12, 20, 30, 40, 50, 51, 60, 63, 65, 67, 68, 69, 70, 71, 72, 73}
{0, 2, 3, 4, 5, 6, 7, 8, 10, 11, 17, 19, 34, 37, 38, 41, 42, 43, 44, 45}


In [325]:
final_rows = []

# Process all source documents in order
for source_idx, source_doc in enumerate(source_docs):
    matched = False
    for (src, tgt) in mappings:
        if src.metadata['line_number'] == source_doc.metadata['line_number'] and src.page_content == source_doc.page_content:
            final_rows.append({
                "source_statement": src.page_content,
                "source_file_name": src.metadata['file_name'],
                "source_line_number": src.metadata['line_number'],
                "target_statement": tgt.page_content,
                "target_file_name": tgt.metadata['file_name'],
                "target_line_number": tgt.metadata['line_number']
            })
            matched = True
            break
    if not matched:
        final_rows.append({
            "source_statement": source_doc.page_content,
            "source_file_name": source_doc.metadata['file_name'],
            "source_line_number": source_doc.metadata['line_number'],
            "target_statement": "no Match Found",
            "target_file_name": "",
            "target_line_number": ""
        })

# Process all target documents in order for unmatched targets
for target_idx, target_doc in enumerate(target_docs):
    if target_idx not in matched_target_indices:
        final_rows.append({
            "source_statement": "no Match Found",
            "source_file_name": "",
            "source_line_number": "",
            "target_statement": target_doc.page_content,
            "target_file_name": target_doc.metadata['file_name'],
            "target_line_number": target_doc.metadata['line_number']
        })

# Create DataFrame
df = pd.DataFrame(final_rows)

In [326]:
df.to_excel("C:\Repos\Agentic_AI\QM_Practice\Langchain\output.xlsx", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Quadrant\AppData\Local\Temp\ipykernel_5940\952741706.py:1: SyntaxWarning: invalid escape sequence '\R'
  df.to_excel("C:\Repos\Agentic_AI\QM_Practice\Langchain\output.xlsx", index=False)
